In [1]:
# Import necessary libraries
from binance.client import Client
import os
from dotenv import load_dotenv # Import the function

# Load environment variables from .env file
load_dotenv() 

# Now access the environment variables
# These names MUST match the names used in your .env file
api_key = os.environ.get('BINANCE_API_KEY') 
api_secret = os.environ.get('BINANCE_API_SECRET')

# Verify if the keys were loaded successfully
print("API Key loaded:", api_key is not None)
print("API Secret loaded:", api_secret is not None) 

# Optional: Print a few characters of the API key to be sure (NEVER print the secret)
if api_key:
    print("API Key starts with:", api_key[:5]) 

API Key loaded: True
API Secret loaded: True
API Key starts with: GVnoN


In [2]:
# Make sure api_key and api_secret are loaded from the previous cell

# Initialize the Binance Client
# IMPORTANT: Use tld='us' for Binance.US
client = Client(api_key, api_secret, tld='us')

# Test the connection with a simple ping
try:
    ping_result = client.ping()
    print("Successfully pinged Binance.US API!")
    # ping_result should be an empty dictionary {} on success
    print("Ping result:", ping_result) 
except Exception as e:
    print(f"Error connecting to Binance.US API: {e}")

Successfully pinged Binance.US API!
Ping result: {}


In [3]:
# Fetch account information (includes balances)
try:
    account_info = client.get_account()
    
    # Print the keys available in the account_info dictionary
    # This helps understand the structure of the response
    print("Available keys in account info:", account_info.keys()) 
    
    # Extract and print balances (usually a list of dictionaries)
    balances = account_info.get('balances', []) 
    print("\n--- Account Balances ---")
    found_assets = False
    for asset in balances:
        # Only print assets with a non-zero balance (free or locked)
        if float(asset['free']) > 0 or float(asset['locked']) > 0:
            print(f"Asset: {asset['asset']}, Free: {asset['free']}, Locked: {asset['locked']}")
            found_assets = True
    
    if not found_assets:
        print("No assets with a balance found (or only zero balances).")
        
except Exception as e:
    print(f"Error fetching account information: {e}")

Available keys in account info: dict_keys(['makerCommission', 'takerCommission', 'buyerCommission', 'sellerCommission', 'commissionRates', 'canTrade', 'canWithdraw', 'canDeposit', 'brokered', 'requireSelfTradePrevention', 'updateTime', 'accountType', 'balances', 'permissions'])

--- Account Balances ---
Asset: BTC, Free: 0.00042710, Locked: 0.00000000
Asset: BUSD, Free: 0.14956400, Locked: 0.00000000
Asset: WAVES, Free: 0.01000000, Locked: 0.00000000


In [4]:
# Define the trading pair symbol
# Symbols on Binance typically combine the base and quote asset without slashes (e.g., BTCUSDT)
symbol = 'BTCUSDT' 

try:
    # Fetch the latest ticker price information for the symbol
    ticker_info = client.get_symbol_ticker(symbol=symbol)
    
    print(f"--- Ticker Information for {symbol} ---")
    print(f"Symbol: {ticker_info.get('symbol')}")
    print(f"Price: {ticker_info.get('price')}") 
    
    # You can print the whole dictionary to see all available info
    # print("\nFull Ticker Info:")
    # print(ticker_info)

except Exception as e:
    print(f"Error fetching ticker information for {symbol}: {e}")
    # Common error: Invalid symbol if the pair doesn't exist on Binance.US

--- Ticker Information for BTCUSDT ---
Symbol: BTCUSDT
Price: 78911.62000000


In [6]:
import pandas as pd

# Define symbol and interval
symbol = 'BTCUSDT'
# Options: KLINE_INTERVAL_1MINUTE, KLINE_INTERVAL_5MINUTE, KLINE_INTERVAL_1HOUR, KLINE_INTERVAL_1DAY, etc.
interval = Client.KLINE_INTERVAL_1HOUR 
limit = 100 # Number of candles to retrieve (max 1000)

try:
    print(f"Fetching last {limit} klines for {symbol} on interval {interval}...")
    # Fetch klines (candlestick data)
    # Returns a list of lists: [open_time, open, high, low, close, volume, close_time, ...]
    klines_raw = client.get_klines(symbol=symbol, interval=interval, limit=limit)

    # Define columns based on Binance API documentation for klines
    columns = [
        'Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 
        'Close Time', 'Quote Asset Volume', 'Number of Trades', 
        'Taker Buy Base Asset Volume', 'Taker Buy Quote Asset Volume', 'Ignore'
    ]
    
    # Create Pandas DataFrame
    klines_df = pd.DataFrame(klines_raw, columns=columns)

    # --- Data Cleaning and Formatting ---
    # Convert timestamp columns to datetime objects (timestamps are in milliseconds)
    klines_df['Open Time'] = pd.to_datetime(klines_df['Open Time'], unit='ms')
    klines_df['Close Time'] = pd.to_datetime(klines_df['Close Time'], unit='ms')

    # Convert relevant numeric columns to float/numeric type
    numeric_cols = ['Open', 'High', 'Low', 'Close', 'Volume', 'Quote Asset Volume', 
                    'Taker Buy Base Asset Volume', 'Taker Buy Quote Asset Volume']
    for col in numeric_cols:
        klines_df[col] = pd.to_numeric(klines_df[col])
        
    # Set 'Open Time' as the index (optional but common)
    # klines_df.set_index('Open Time', inplace=True)

    print(f"\n--- Last 5 Klines for {symbol} ({interval}) ---")
    # Display the last 5 rows of the DataFrame
    print(klines_df.tail()) 

    print(f"\nDataFrame shape: {klines_df.shape}") # Should show (limit, 12) e.g. (100, 12)

except Exception as e:
    print(f"Error fetching klines for {symbol}: {e}")

Fetching last 100 klines for BTCUSDT on interval 1h...

--- Last 5 Klines for BTCUSDT (1h) ---
             Open Time      Open      High       Low     Close   Volume  \
95 2025-04-07 14:00:00  78160.53  81188.05  77965.44  79000.09  5.87123   
96 2025-04-07 15:00:00  79000.00  79337.51  77652.61  78790.54  1.79442   
97 2025-04-07 16:00:00  78496.71  78497.82  77362.16  77362.16  0.93518   
98 2025-04-07 17:00:00  77520.28  79101.14  77520.28  79051.60  0.23073   
99 2025-04-07 18:00:00  79101.08  79305.61  78541.16  78911.62  0.50797   

                Close Time  Quote Asset Volume  Number of Trades  \
95 2025-04-07 14:59:59.999       465379.411499              1005   
96 2025-04-07 15:59:59.999       140580.766465               292   
97 2025-04-07 16:59:59.999        73191.965486               199   
98 2025-04-07 17:59:59.999        18090.991717               170   
99 2025-04-07 18:59:59.999        40129.037689                96   

    Taker Buy Base Asset Volume  Taker Buy Qu

In [7]:
# Cell 2: Function to Fetch and Process Klines

def fetch_and_process_klines(api_client, symbol, interval, start_date_str, end_date_str=None):
    """
    Fetches historical klines for a symbol/interval/date range, processes
    them into a pandas DataFrame, and handles potential API limits.

    Args:
        api_client: Initialized Binance Client.
        symbol (str): The trading symbol (e.g., 'BTCUSDT').
        interval (str): Kline interval (e.g., Client.KLINE_INTERVAL_1HOUR).
        start_date_str (str): Start date string (e.g., "1 day ago UTC", "1 Jan, 2023").
        end_date_str (str, optional): End date string. Defaults to None (fetches up to now).

    Returns:
        pandas.DataFrame: DataFrame with processed kline data, or None if error.
                          Indexed by 'Open Time'. Columns include OHLCV etc.
    """
    if not api_client:
        print("API Client not available.")
        return None

    print(f"Fetching klines for {symbol}, interval {interval}, starting {start_date_str}...")
    
    try:
        # Use get_historical_klines_generator for automatic pagination
        klines_generator = api_client.get_historical_klines_generator(
            symbol,
            interval,
            start_date_str,
            end_str=end_date_str
            # limit=1000 # Default chunk size
        )
        
        # Convert generator to list (can consume memory for very long ranges)
        all_klines = list(klines_generator)

        if not all_klines:
            print(f"No kline data found for {symbol} with given parameters.")
            return pd.DataFrame() # Return empty DataFrame

        print(f"Fetched {len(all_klines)} klines.")

        # Define columns based on API documentation
        columns = [
            'Open Time', 'Open', 'High', 'Low', 'Close', 'Volume',
            'Close Time', 'Quote Asset Volume', 'Number of Trades',
            'Taker Buy Base Asset Volume', 'Taker Buy Quote Asset Volume', 'Ignore'
        ]

        # Create DataFrame
        klines_df = pd.DataFrame(all_klines, columns=columns)

        # --- Data Cleaning ---
        klines_df['Open Time'] = pd.to_datetime(klines_df['Open Time'], unit='ms', utc=True) # Add UTC timezone
        klines_df['Close Time'] = pd.to_datetime(klines_df['Close Time'], unit='ms', utc=True) # Add UTC timezone

        numeric_cols = ['Open', 'High', 'Low', 'Close', 'Volume', 'Quote Asset Volume',
                        'Taker Buy Base Asset Volume', 'Taker Buy Quote Asset Volume']
        for col in numeric_cols:
            klines_df[col] = pd.to_numeric(klines_df[col])

        # Drop the 'Ignore' column
        klines_df.drop('Ignore', axis=1, inplace=True)

        # Set 'Open Time' as the index
        klines_df.set_index('Open Time', inplace=True)
        
        print(f"Successfully processed klines for {symbol} ({interval}).")
        return klines_df

    except Exception as e:
        print(f"An error occurred fetching/processing klines for {symbol} ({interval}): {e}")
        return None

print("Kline fetching function defined.")

Kline fetching function defined.


In [8]:
# Cell 3: Fetch Data for Multiple Intervals

symbol_to_analyze = 'BTCUSDT'
intervals_to_fetch = {
    '1d': Client.KLINE_INTERVAL_1DAY,
    '1h': Client.KLINE_INTERVAL_1HOUR,
    '5m': Client.KLINE_INTERVAL_5MINUTE,
    '1m': Client.KLINE_INTERVAL_1MINUTE 
}

# Define how far back to fetch for different interval types
# More recent data for shorter intervals
start_dates = {
    '1d': "90 days ago UTC", # ~3 months of daily data
    '1h': "7 days ago UTC",  # 1 week of hourly data
    '5m': "3 days ago UTC",  # 3 days of 5-min data
    '1m': "1 day ago UTC"   # 1 day of 1-min data
}

klines_data = {} # Dictionary to store DataFrames for each interval

if client:
    for interval_key, interval_value in intervals_to_fetch.items():
        start_str = start_dates.get(interval_key, "7 days ago UTC") # Default if key missing
        df = fetch_and_process_klines(client, symbol_to_analyze, interval_value, start_str)
        
        if df is not None and not df.empty:
            klines_data[interval_key] = df
            print(f"Stored data for {interval_key}. Shape: {df.shape}")
            print(f"  Latest entry: {df.index[-1]}") # Show timestamp of last candle
            print("-" * 30)
        else:
            print(f"Failed to fetch or no data for {interval_key}.")
            print("-" * 30)
            
    print("\nFinished fetching multi-interval data.")
    print(f"Available dataframes: {list(klines_data.keys())}")

else:
    print("Client not initialized. Cannot fetch data.")

# --- Display Sample Data (Optional) ---
# Example: Display last 5 rows of the 5-minute data if available
if '5m' in klines_data:
     print("\n--- Sample 5m Data (Last 5 rows) ---")
     print(klines_data['5m'].tail())

Fetching klines for BTCUSDT, interval 1d, starting 90 days ago UTC...
Fetched 90 klines.
Successfully processed klines for BTCUSDT (1d).
Stored data for 1d. Shape: (90, 10)
  Latest entry: 2025-04-07 00:00:00+00:00
------------------------------
Fetching klines for BTCUSDT, interval 1h, starting 7 days ago UTC...
Fetched 168 klines.
Successfully processed klines for BTCUSDT (1h).
Stored data for 1h. Shape: (168, 10)
  Latest entry: 2025-04-07 18:00:00+00:00
------------------------------
Fetching klines for BTCUSDT, interval 5m, starting 3 days ago UTC...
Fetched 864 klines.
Successfully processed klines for BTCUSDT (5m).
Stored data for 5m. Shape: (864, 10)
  Latest entry: 2025-04-07 18:45:00+00:00
------------------------------
Fetching klines for BTCUSDT, interval 1m, starting 1 day ago UTC...
Fetched 1440 klines.
Successfully processed klines for BTCUSDT (1m).
Stored data for 1m. Shape: (1440, 10)
  Latest entry: 2025-04-07 18:48:00+00:00
------------------------------

Finished fe

In [9]:
# Cell 3 (Modified): Fetch Data for Multiple Intervals AND SAVE

symbol_to_analyze = 'BTCUSDT'
intervals_to_fetch = {
    '1d': Client.KLINE_INTERVAL_1DAY,
    '1h': Client.KLINE_INTERVAL_1HOUR,
    '5m': Client.KLINE_INTERVAL_5MINUTE,
    '1m': Client.KLINE_INTERVAL_1MINUTE
}

# Define how far back to fetch for different interval types
start_dates = {
    '1d': "90 days ago UTC", 
    '1h': "7 days ago UTC",  
    '5m': "3 days ago UTC",  
    '1m': "1 day ago UTC"   
}

klines_data = {} # Dictionary to store DataFrames
data_directory = "data" # Make sure this directory exists

# Ensure data directory exists
os.makedirs(data_directory, exist_ok=True)


if client:
    print("--- Starting Multi-Interval Data Fetch and Save ---")
    for interval_key, interval_value in intervals_to_fetch.items():
        start_str = start_dates.get(interval_key, "7 days ago UTC") 
        print("-" * 30) # Separator for clarity
        
        # Call the fetching function (defined in Cell 2)
        df = fetch_and_process_klines(client, symbol_to_analyze, interval_value, start_str)
        
        if df is not None and not df.empty:
            klines_data[interval_key] = df # Store in memory dictionary
            
            # --- SAVE TO CSV ---
            # Create a filename specific to this interval and approximate date range
            # Use replace for file system compatibility
            start_date_safe = start_str.replace(' ','_').replace(',','') 
            filename = os.path.join(data_directory, f"{symbol_to_analyze}_{interval_key}_{start_date_safe}.csv")
            
            try:
                df.to_csv(filename) # Save the dataframe (index is included by default)
                print(f"Data for {interval_key} SAVED to: {filename}")
                print(f"  Shape: {df.shape}, Latest entry: {df.index[-1]}") 
            except Exception as e:
                print(f"!!! Error saving data for {interval_key} to {filename}: {e}")
            # --- END SAVE TO CSV ---

        else:
            print(f"Failed to fetch or no data for {interval_key}.")
            
    print("-" * 30)
    print("\nFinished fetching multi-interval data.")
    print(f"DataFrames stored in memory: {list(klines_data.keys())}")
    print(f"Data also saved to individual CSV files in '{data_directory}' folder.")

else:
    print("Client not initialized. Cannot fetch data.")

# --- Display Sample Data (Optional) ---
if '1m' in klines_data: # Example: Show 1-minute data tail
     print("\n--- Sample 1m Data (Last 5 rows) ---")
     print(klines_data['1m'].tail())

--- Starting Multi-Interval Data Fetch and Save ---
------------------------------
Fetching klines for BTCUSDT, interval 1d, starting 90 days ago UTC...
Fetched 90 klines.
Successfully processed klines for BTCUSDT (1d).
Data for 1d SAVED to: data/BTCUSDT_1d_90_days_ago_UTC.csv
  Shape: (90, 10), Latest entry: 2025-04-07 00:00:00+00:00
------------------------------
Fetching klines for BTCUSDT, interval 1h, starting 7 days ago UTC...
Fetched 168 klines.
Successfully processed klines for BTCUSDT (1h).
Data for 1h SAVED to: data/BTCUSDT_1h_7_days_ago_UTC.csv
  Shape: (168, 10), Latest entry: 2025-04-07 18:00:00+00:00
------------------------------
Fetching klines for BTCUSDT, interval 5m, starting 3 days ago UTC...
Fetched 864 klines.
Successfully processed klines for BTCUSDT (5m).
Data for 5m SAVED to: data/BTCUSDT_5m_3_days_ago_UTC.csv
  Shape: (864, 10), Latest entry: 2025-04-07 18:50:00+00:00
------------------------------
Fetching klines for BTCUSDT, interval 1m, starting 1 day ago U

In [1]:
# Cell X (Diagnostics): Live Ticker WebSocket Stream with Diagnostics

import asyncio
from binance import AsyncClient, BinanceSocketManager
from dotenv import load_dotenv
import os
import nest_asyncio
import signal 
import time # For simple loop counter

# Apply the patch
nest_asyncio.apply()
print("nest_asyncio applied.")

# --- Global flag and Signal Handler (Keep as before) ---
shutdown_flag = False
def handle_signal(sig, frame):
    global shutdown_flag
    print("\nInterrupt signal received, setting shutdown flag...")
    shutdown_flag = True
signal.signal(signal.SIGINT, handle_signal)
signal.signal(signal.SIGTERM, handle_signal)
print("Signal handlers registered.")


# --- Function to handle incoming messages (Keep as corrected before) ---
async def handle_ticker_message(msg):
    if msg and msg.get('e') == '24hrTicker': 
        symbol = msg.get('s')
        last_price = msg.get('c')
        volume = msg.get('v')
        if symbol and last_price and volume: 
            print(f"\rTicker Update: {symbol} | Price: {last_price} | 24h Vol: {volume}    ", end='')
    # else: # Optional: Print unexpected messages if debugging needed
    #    print(f"\nNon-ticker or unexpected message format: {msg}")


# --- Main function to start the WebSocket ---
async def main_websocket_loop(api_key, api_secret):
    global shutdown_flag 
    print("Initializing Asynchronous Client...")
    client = await AsyncClient.create(api_key, api_secret, tld='us')
    print("Initializing Binance Socket Manager...")
    bsm = BinanceSocketManager(client)
    symbol = 'BTCUSDT'
    print(f"Starting WebSocket connection for stream: {symbol.lower()}@ticker")
    
    loop_count = 0 # Add a counter
    
    try:
        async with bsm.symbol_ticker_socket(symbol=symbol) as ts:
            while not shutdown_flag:
                loop_count += 1
                print(f"\rLoop {loop_count}: Waiting for message...{' '*20}", end='') # Print loop status
                try:
                    res = await asyncio.wait_for(ts.recv(), timeout=1.0)
                    # --- DIAGNOSTIC PRINT ---
                    # Temporarily print the raw message received WITHOUT \r to see if multiple messages arrive
                    print(f"\nLoop {loop_count}: Received message: {res}\n") 
                    # --- END DIAGNOSTIC ---
                    await handle_ticker_message(res) # Process message (this will overwrite with \r if it runs)
                except asyncio.TimeoutError:
                    # This is expected if no message in 1s, just continue loop
                    print(f"\rLoop {loop_count}: Timeout waiting for message...{' '*10}", end='') 
                    continue 
                except asyncio.CancelledError:
                    print("\nWebSocket task cancelled internally.")
                    shutdown_flag = True 
                    break
                except Exception as e:
                    print(f"\nWebSocket Error: {e}")
                    await asyncio.sleep(5)
            print("\nShutdown flag set, exiting loop.")
    finally:
        print("\nClosing Asynchronous Client session...")
        await client.close_connection()
        print("Client closed.")

# --- Load Keys and Run the Main Loop ---
load_dotenv()
api_key = os.environ.get('BINANCE_API_KEY')
api_secret = os.environ.get('BINANCE_API_SECRET')

if not api_key or not api_secret:
    print("API Key or Secret not found. Cannot start WebSocket.")
else:
    print("Starting WebSocket loop... (Use Kernel -> Interrupt to stop)")
    shutdown_flag = False
    loop = asyncio.get_event_loop()
    try:
        loop.run_until_complete(main_websocket_loop(api_key, api_secret))
    except KeyboardInterrupt:
        print("\nKeyboardInterrupt caught.")
    except Exception as e:
         print(f"\nError running main loop: {e}")
    finally:
        print("Main loop finished or interrupted.")
        shutdown_flag = True

nest_asyncio applied.
Signal handlers registered.
Starting WebSocket loop... (Use Kernel -> Interrupt to stop)
Initializing Asynchronous Client...
Initializing Binance Socket Manager...
Starting WebSocket connection for stream: btcusdt@ticker
Loop 1: Waiting for message...                    
Loop 1: Received message: {'e': '24hrTicker', 'E': 1744052307232, 's': 'BTCUSDT', 'p': '-590.00000000', 'P': '-0.746', 'w': '77485.84623204', 'x': '79101.14000000', 'c': '78497.82000000', 'Q': '0.00384000', 'b': '78460.92000000', 'B': '0.16846000', 'a': '78591.40000000', 'A': '0.63392000', 'o': '79087.82000000', 'h': '81188.05000000', 'l': '74557.35000000', 'v': '43.52255000', 'q': '3372381.61692630', 'O': 1743965907232, 'C': 1744052307232, 'F': 30381898, 'L': 30387949, 'n': 6052}

Loop 3: Waiting for message...                    4h Vol: 43.52255000    
Loop 3: Received message: {'e': '24hrTicker', 'E': 1744052308396, 's': 'BTCUSDT', 'p': '-590.00000000', 'P': '-0.746', 'w': '77485.84623204', 'x'